In [1]:
#Code for dummy disease map. Generates lung mask with random values corresponding to diseases.
import numpy as np

# Define the dimensions of the 3D array
dimensions = (200, 200, 200)

# Define the values and their corresponding probabilities
values = [0, 2, 3, 4]
probabilities = [0.1, 0.7, 0.1, 0.1]

# Create the 3D array with random values based on probabilities
random_values = np.random.choice(values, size=dimensions, p=probabilities)

# Now, 'random_values' is a 3D NumPy array with the specified dimensions and values sampled based on probabilities.

In [2]:
#Function from Nate's code
def update_camera_info(obj, evt):
                    camera = mlab.gcf().scene.camera
                    print("Camera Position:", camera.position)
                    print("Camera Focal Point:", camera.focal_point)
                    print("Camera View Up:", camera.view_up)

In [3]:
#Load in the file
import nrrd
import nibabel as nib
from mayavi import mlab

# Load the NIfTI image using nibabel
file_path = r'Z:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done\Case-50-Spiromics-45626090\seg_net\SPI-IA210305-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz'
nii_image = nib.load(file_path)

In [4]:
# Get the image data as a NumPy array
image_data = nii_image.get_fdata().astype(int)
# image_data = np.where(image_data != 0, 1, image_data)
condition = image_data > 0
image_data[condition] = 1

#Create lung pathogen map
dimensions = image_data.shape

# Define the values and their corresponding probabilities
values = [0, 2, 3, 4]
probabilities = [1-0.06, 0.02, 0.02, 0.02]

# Create the 3D array with random values based on probabilities
diseaseEX = np.random.choice(values, size=dimensions, p=probabilities)
diseaseEX = diseaseEX * image_data

#Center of mass coordinates
coords = np.argwhere(image_data)
center_of_mass = np.mean(coords, axis=0)
center_of_mass = (center_of_mass / 10).astype(int)   #Remove /10         #center_of_mass.astype(int) / 10
image_data[*center_of_mass] = 2

In [5]:
import numpy as np

# Define the dimensions of the array
shape = (512, 512, 556)

# Create an empty array filled with zeros
array = np.zeros(shape)

# Define the center and cutoff distance
center = (240, 280, 300)
cutoff_distance = 100

# Create coordinate grids for each dimension
x, y, z = np.meshgrid(
    np.arange(shape[0]),
    np.arange(shape[1]),
    np.arange(shape[2])
)

# Calculate the Euclidean distance from the center for each point
distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)

# Apply the Gaussian distribution with a mean of 0 and standard deviation of 1
gaussian = np.exp(-0.5 * (distance / cutoff_distance)**2)

# Threshold the Gaussian values to either 0 or 2
array[gaussian >= 0.9] = 2

# Print a slice of the array to verify
print(array[:, :, 250])  # Print a slice along the z-axis (at z=250)

array = array * image_data

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
array.max()

0.0

In [7]:
from sklearn.mixture import GaussianMixture
N = 3
array = np.zeros((20,20,20))
array[0:8, 0:8, 0:8] = 2
array[16:20, 16:20, 0:4] = 2
array[0:4, 16:20, 16:20] = 2
indices = np.argwhere(array == 2)

# disease_data = array
# disease_data = disease_data.reshape(-1, 3)
# dSampDisease = disease_data[::10, ::10]
gmm = GaussianMixture(n_components=N)
gmm.fit(indices)

gmm.means_
np.sqrt(np.abs(gmm.covariances_))  # Take the square root to get standard deviations (sigma values)
array.max()


2.0

In [8]:
# Create a figure
mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white

# Visualize the entire image as a volume (adjust opacity as needed)
dummyObj = np.ones((20,20,20))
dummy = mlab.contour3d(dummyObj, contours=[1.0], opacity=0.15, color=(128/255, 174/255, 128/255))
disease = mlab.contour3d(array, contours=[1.5], opacity = 0.9, color = (1, 0, 0))
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

mlab.view(azimuth=90, elevation=90, distance=600/0.546875, focalpoint=center_of_mass)
mlab.gcf().scene.camera.view_angle = 23

In [9]:
for i in indices:
    print(i)

[0 0 0]
[0 0 1]
[0 0 2]
[0 0 3]
[0 0 4]
[0 0 5]
[0 0 6]
[0 0 7]
[0 1 0]
[0 1 1]
[0 1 2]
[0 1 3]
[0 1 4]
[0 1 5]
[0 1 6]
[0 1 7]
[0 2 0]
[0 2 1]
[0 2 2]
[0 2 3]
[0 2 4]
[0 2 5]
[0 2 6]
[0 2 7]
[0 3 0]
[0 3 1]
[0 3 2]
[0 3 3]
[0 3 4]
[0 3 5]
[0 3 6]
[0 3 7]
[0 4 0]
[0 4 1]
[0 4 2]
[0 4 3]
[0 4 4]
[0 4 5]
[0 4 6]
[0 4 7]
[0 5 0]
[0 5 1]
[0 5 2]
[0 5 3]
[0 5 4]
[0 5 5]
[0 5 6]
[0 5 7]
[0 6 0]
[0 6 1]
[0 6 2]
[0 6 3]
[0 6 4]
[0 6 5]
[0 6 6]
[0 6 7]
[0 7 0]
[0 7 1]
[0 7 2]
[0 7 3]
[0 7 4]
[0 7 5]
[0 7 6]
[0 7 7]
[ 0 16 16]
[ 0 16 17]
[ 0 16 18]
[ 0 16 19]
[ 0 17 16]
[ 0 17 17]
[ 0 17 18]
[ 0 17 19]
[ 0 18 16]
[ 0 18 17]
[ 0 18 18]
[ 0 18 19]
[ 0 19 16]
[ 0 19 17]
[ 0 19 18]
[ 0 19 19]
[1 0 0]
[1 0 1]
[1 0 2]
[1 0 3]
[1 0 4]
[1 0 5]
[1 0 6]
[1 0 7]
[1 1 0]
[1 1 1]
[1 1 2]
[1 1 3]
[1 1 4]
[1 1 5]
[1 1 6]
[1 1 7]
[1 2 0]
[1 2 1]
[1 2 2]
[1 2 3]
[1 2 4]
[1 2 5]
[1 2 6]
[1 2 7]
[1 3 0]
[1 3 1]
[1 3 2]
[1 3 3]
[1 3 4]
[1 3 5]
[1 3 6]
[1 3 7]
[1 4 0]
[1 4 1]
[1 4 2]
[1 4 3]
[1 4 4]
[1 4 5]
[1 4 6]


: 

In [8]:
# Create a figure
mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white

# Visualize the entire image as a volume (adjust opacity as needed)
lungs = mlab.contour3d(image_data[::10,::10,::10], contours=[0.5], opacity = 0.15, color = (128/255, 174/255, 128/255))
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

#Dummy Lung Mask
obj = (array == 2).astype(int)
mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (1, 0, 0))   #Remove later
# obj = (diseaseEX == 3).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (0, 0, 1))
# obj = (diseaseEX == 4).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (148/255, 0, 211/255))

# Set the camera position and direction to focus on the center of mass
mlab.view(azimuth=90, elevation=90, distance=600/0.546875, focalpoint=(0,0,0))
mlab.gcf().scene.camera.view_angle = 23

#Saving file
outputPath = r"z:/Q- Figures/T32-Poster/Nick/Experiment/"
mlab.savefig(outputPath + "test1.png")   #, magnification=5     <-- Add argument for magnification factor

# Show the visualization
mlab.show()

Exception occurred in traits notification handler for object: <mayavi.tools.modules.IsoSurfaceFactory object at 0x00000240AC042390>, trait: contours, old value: 5, new value: [0.5]
Traceback (most recent call last):
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\mayavi\tools\modules.py", line 185, in _contours_changed
    self._target.contour.contours = self.contours
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_types.py", line 2695, in validate
    return TraitListObject(self, object, name, value)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_list_object.py", line 584, in __init__
    super().__init__(
  File "

TraitError: Each element of the 'contours' trait of a Contour instance must be 0.0 <= a number <= 0.0, but a value of 0.5 <class 'float'> was specified.

In [9]:
image_data.shape

(512, 512, 556)